In [1]:
import pandas as pd
import numpy as np
import random

#split dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#AUCROC,AUCPR,precision,recall,f1-score
from sklearn.metrics import roc_curve,auc,average_precision_score,precision_score,recall_score,f1_score

#gridsearch/randomsearch
from itertools import product
from tqdm import tqdm

#visualize results
import matplotlib.pyplot as plt
import time

In [2]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Subset,DataLoader,TensorDataset
from torchvision import datasets,transforms
import torch.nn.functional as F

In [3]:
# import warnings
# warnings.filterwarnings("ignore")

if torch.cuda.is_available():
  is_cuda=True
  print('GPU is on')
else:
  is_cuda=False
  print('GPU is off')

GPU is off


In [4]:
def set_seed(seed):
  np.random.seed(seed)
  torch.manual_seed(seed)
  random.seed(seed)

  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [5]:
def data_prepare(batch_size,contam_ratio,hybrid,seed):
    set_seed(seed)
    data = pd.read_csv("D:/Jiang/Research_Anomaly Detection/Important_Credit Card Fraud Detection (CCFD)/creditcardfraud/creditcard.csv")
    #change 0,1 label to 1,-1
    data.loc[data['Class']==1,'Class'] = -1
    data.loc[data['Class']==0,'Class'] = 1

    X = data.drop(['Time','Class'], axis=1)
    y = data["Class"].values

    #split the data to training, validation and testing data (50%,20%,30%)
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,shuffle=False)
    X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=2/7,shuffle=False)

    #the known positive samples before contaminating
    known_pos_entire = sum(y_train == -1)
    #Minmax
    scaler=MinMaxScaler().fit(X_train)

    X_train = scaler.transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    index_contam = np.arange(len(y_train))[y_train == -1]
    index_contam = np.random.choice(index_contam,int(contam_ratio*len(index_contam)),replace = False)

    y_train[index_contam] = 1
    #the known positive samples after contaminating
    known_pos_sub = sum(y_train == -1)

    print(f'The left true(known) positive samples in the training set:{known_pos_sub}/{known_pos_entire}\n')
    #normal training tensor and dataloader
    #none-hybrid model only use "normal" samples in the training phase, which could be contaminated
    if not hybrid:
        index_subset = np.arange(len(y_train))[y_train == 1]
    else:
        index_subset = np.arange(len(y_train))
    

    #transform numpy to pytorch tensor
    train_tensor = TensorDataset(torch.from_numpy(X_train).float(),torch.tensor(y_train))
    train_tensor = Subset(train_tensor,index_subset)
    #fitting by batches (using dataloader), there exists randomness when shuffle=True***
    train_loader=DataLoader(train_tensor,batch_size=batch_size,shuffle=False,drop_last=True)
    
    #validation set
    X_val_tensor = torch.from_numpy(X_val).float()
    #testing set
    X_test_tensor=torch.from_numpy(X_test).float()
    
    return train_loader,X_val_tensor,y_val,X_test_tensor,y_test

In [6]:
class aeplus(nn.Module):
  def __init__(self,input_size,act_fun):
    super(aeplus,self).__init__()

    self.encoder_1=nn.Sequential(
      nn.Linear(input_size,input_size//4),
      act_fun,
      )
    
    self.decoder_1=nn.Sequential(
      nn.Linear(input_size//4,input_size),
      )

    self.encoder_2=nn.Sequential(
      nn.Linear(input_size,input_size//4),
      act_fun,
      )

  def forward(self,input):
    z = self.encoder_1(input)
    X_hat = self.decoder_1(z)
    z_hat = self.encoder_2(X_hat)

    return z,X_hat,z_hat

In [7]:
def fit(dataloader,model_aeplus,epochs,batch_size,print_loss):
  L1_criterion = nn.L1Loss(reduction='mean')

  if is_cuda:
    L1_criterion.cuda()

  loss_list = []

  for epoch in range(epochs):
    for i,data in enumerate(dataloader):
      X,_ = data
      if is_cuda:
        X = X.cuda()
      X = Variable(X)
        
      model_aeplus.zero_grad()

      z,X_hat,z_hat = model_aeplus(X)

      #contextual loss
      loss_contextual = L1_criterion(X,X_hat)
      #encoder loss
      loss_encoder = L1_criterion(z,z_hat)

      loss = (loss_contextual + loss_encoder)/2

      #backward and upgrade
      loss.backward()
      optimizer.step() 

      if (i%50 == 0) & print_loss:
        print('[%d/%d] [%d/%d] Loss: %.4f' % (epoch+1,epochs,i,len(dataloader),loss))

      loss_list.append(loss)

In [8]:
def evaluation(data_tensor,model):
  if is_cuda:
    data_tensor = data_tensor.cuda()
    
  z,_,z_hat = model(data_tensor)

  L1_criterion = nn.L1Loss(reduction='none')
  score = L1_criterion(z,z_hat)
  score = torch.sum(score,dim=1).cpu().detach().numpy()
    
  return score

Global Parameters

In [9]:
seed_pool = [1,2,3,4,5]
anomaly_ratio_pool = [0.001,0.002,0.003]
contam_ratio_pool = [1.0,0.98,0.8,0.5,0.0]

#random search size
search_size = 10

In [10]:
epochs = 20
batch_size = 64

hyper_act_fun = [nn.Tanh(),nn.LeakyReLU()]
hyper_lr = [1e-2,1e-3,1e-4]
hyper_mom = [0.5,0.7,0.9]

hyper_list_entire = list(product(hyper_act_fun,hyper_lr,hyper_mom))

In [11]:
def random_search(hyper_list_entire,search_size,seed):
  if search_size < len(hyper_list_entire):
    set_seed(seed)
    index = np.random.choice(np.arange(len(hyper_list_entire)),search_size,replace = False)

    hyper_list = []
    for i in index:
      hyper_list.append(hyper_list_entire[i])
  else:
    hyper_list = hyper_list_entire

  return hyper_list

In [12]:
for contam_ratio in tqdm(contam_ratio_pool):
  df_result = pd.DataFrame(data = None,index = ['AUCPR'] + anomaly_ratio_pool,columns = seed_pool)
  for seed in tqdm(seed_pool):
    #############################################seleting the best hyper-parameters in validation set#############################################
    metric_value_list=list()
    hyper_list = random_search(hyper_list_entire,search_size,seed)
    for i in range(len(hyper_list)):
      try:
        print(f'Finding Optimal Hyper-parameters......Current Candidates: {hyper_list[i]}')
        act_fun,lr,mom = hyper_list[i]
        #data
        train_loader,X_val_tensor,y_val,_,_ = data_prepare(batch_size,contam_ratio,hybrid = False,seed = seed)
        #model initialization
        set_seed(seed)
        model_aeplus = aeplus(input_size = X_val_tensor.size(1),act_fun = act_fun)

        if is_cuda:
          model_aeplus.cuda()

        optimizer = torch.optim.SGD(model_aeplus.parameters(),lr = lr,momentum = mom)
        #fitting
        fit(dataloader = train_loader,model_aeplus = model_aeplus,epochs = epochs,batch_size = batch_size,print_loss = False)
        
        #evaluation
        score = evaluation(data_tensor = X_val_tensor,model = model_aeplus)
        metric_value = average_precision_score(y_true = y_val,y_score = score,pos_label = -1)
        metric_value_list.append(metric_value)

        print(f'The metric value corresponded to the hyper-parameters is :{metric_value:{.4}}')
        print('******************************')
        print('\n')
      except:
        #keep the right index
        metric_value_list.append(0)
        pass
      continue

    best_hyper_params=hyper_list[metric_value_list.index(max(metric_value_list))]
    print(f'The best hyper-parameters are: {best_hyper_params}')
    print('\n')
    ###################################################################testing#########################################################################
    print('Testing Phrase......')
    act_fun,lr,mom = best_hyper_params

    #data
    train_loader,_,_,X_test_tensor,y_test = data_prepare(batch_size,contam_ratio,hybrid = False,seed = seed)
    
    #model initialization, there exists randomness because of weight initialization***
    set_seed(seed)
    model_aeplus = aeplus(input_size = X_val_tensor.size(1),act_fun = act_fun)

    if is_cuda:
      model_aeplus.cuda()

    optimizer = torch.optim.SGD(model_aeplus.parameters(),lr = lr,momentum = mom)
    #fitting
    fit(dataloader = train_loader,model_aeplus = model_aeplus,epochs = epochs,batch_size = batch_size,print_loss = False)
    #evaluation
    score = evaluation(data_tensor = X_test_tensor,model = model_aeplus)
    
    #store the result
    #AUCPR
    df_result.loc['AUCPR',seed] = average_precision_score(y_true = y_test,y_score = score,pos_label = -1)
    #F1
    for anomaly_ratio in anomaly_ratio_pool:
        threshold = score[np.argsort(-score)][int(anomaly_ratio*len(score))]
        
        y_pred = np.ones(len(score))
        y_pred[score >= threshold] = -1
        
        print('\n')
        print(f'Precision: {round(precision_score(y_pred = y_pred, y_true = y_test, pos_label= -1)*100,2)}')
        print(f'Recall: {round(recall_score(y_pred = y_pred, y_true = y_test, pos_label= -1)*100,2)}')
        print(f'F1-score: {round(f1_score(y_pred = y_pred, y_true = y_test, pos_label= -1)*100,2)}')
        print('\n')

        df_result.loc[anomaly_ratio,seed] = f1_score(y_pred = y_pred,y_true = y_test,pos_label = -1)

  #mean & sd
  df_result['mean'] = np.mean(df_result.loc[:,seed_pool],axis = 1)
  df_result['std'] = np.std(df_result.loc[:,seed_pool],axis = 1)
  df_result = round(df_result.astype('float64')*100,2)

  file_path = 'D:/Jiang/Research_Anomaly Detection/Important_Credit Card Fraud Detection (CCFD)/Hyprid Semi-supervised/result/' + 'CCFD_AEplus_' + str(contam_ratio) + '.csv'
  df_result.to_csv(file_path,index = False)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.0001, 0.5)
The left true(known) positive samples in the training set:0/269

The metric value corresponded to the hyper-parameters is :0.2817
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.001, 0.5)
The left true(known) positive samples in the training set:0/269

The metric value corresponded to the hyper-parameters is :0.5506
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.001, 0.7)
The left true(known) positive samples in the training set:0/269

The metric value corresponded to the hyper-parameters is :0.6125
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.01, 0.9)
The left true(known) positive samples in the training set:0/269

The metric value corresponded to the hyper-parameters is :0.7458
******************************


Finding 


 20%|████████████████▌                                                                  | 1/5 [12:31<50:06, 751.74s/it]

Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.01, 0.5)
The left true(known) positive samples in the training set:0/269

The metric value corresponded to the hyper-parameters is :0.3683
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.001, 0.7)
The left true(known) positive samples in the training set:0/269

The metric value corresponded to the hyper-parameters is :0.5195
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.0001, 0.7)
The left true(known) positive samples in the training set:0/269

The metric value corresponded to the hyper-parameters is :0.3399
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.01, 0.5)
The left true(known) positive samples in the training set:0/269

The metric value corresponded to the hyper-parameters is :0.661
*******************


 40%|█████████████████████████████████▏                                                 | 2/5 [24:57<37:30, 750.08s/it]

...


 60%|█████████████████████████████████████████████████▊                                 | 3/5 [33:11<22:29, 674.88s/it]

Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.0001, 0.5)
The left true(known) positive samples in the training set:269/269

The metric value corresponded to the hyper-parameters is :0.4101
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.001, 0.5)
The left true(known) positive samples in the training set:269/269

The metric value corresponded to the hyper-parameters is :0.6127
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.0001, 0.7)
The left true(known) positive samples in the training set:269/269

The metric value corresponded to the hyper-parameters is :0.567
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.01, 0.9)
The left true(known) positive samples in the training set:269/269

The metric value corresponded to the hyper-parameters is :0.6251
*********


 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [43:52<11:04, 664.81s/it]

Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.001, 0.9)
The left true(known) positive samples in the training set:269/269

The metric value corresponded to the hyper-parameters is :0.7466
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.01, 0.7)
The left true(known) positive samples in the training set:269/269

The metric value corresponded to the hyper-parameters is :0.7674
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.01, 0.7)
The left true(known) positive samples in the training set:269/269

The metric value corresponded to the hyper-parameters is :0.7523
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.01, 0.9)
The left true(known) positive samples in the training set:269/269

The metric value corresponded to the hyper-parameters is :0.7753
******************************


Fin


100%|████████████████████████████████████████████████████████████████████████████████| 5/5 [5:55:02<00:00, 4035.96s/it]
